In [1]:
import sys
import nltk
import sklearn
import pandas
import numpy

print('Python: {}'.format(sys.version))
print('NLTK: {}'.format(nltk.__version__))
print('Scikit-learn: {}'.format(sklearn.__version__))
print('Pandas: {}'.format(pandas.__version__))
print('Numpy: {}'.format(numpy.__version__))

Python: 3.6.3 (default, Oct  3 2017, 21:45:48) 
[GCC 7.2.0]
NLTK: 3.4.4
Scikit-learn: 0.21.3
Pandas: 0.25.0
Numpy: 1.17.0


In [2]:
# Carregando dados
import pandas as pd

df = pd.read_csv('../../data/big/dataset_multi.txt', header=None, encoding='utf-8', sep='\t')
df = df.dropna()
df = df.groupby(by=0).apply(lambda x: x.sample(df.groupby(by=0).size().min()))
df = df.sample(frac=1)

print(df.info())
print(df.head())


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 9740 entries, (constitucional, 213470) to (outros, 646757)
Data columns (total 2 columns):
0    9740 non-null object
1    9740 non-null object
dtypes: object(2)
memory usage: 256.9+ KB
None
                                    0  \
0                                       
constitucional 213470  constitucional   
administrativo 993055  administrativo   
               183316  administrativo   
ambiental      920317       ambiental   
constitucional 802969  constitucional   

                                                                       1  
0                                                                         
constitucional 213470  nao ha qualqu inconstitucional formal lei n in...  
administrativo 993055  fat autenticaco banc nao ser perfeit legiv tod...  
               183316  aca mandamental pressupo prov preconstitu nao ...  
ambiental      920317  crim pesc proib lei art inci depend norm suple...  
constitucional 802969 

In [3]:
# verificação das classes dos acórdãos

classes = df[0]
print(len(classes.value_counts()))
classes.value_counts().to_frame().style.bar()

10


,0
civil,974
administrativo,974
tributário,974
constitucional,974
penal,974
outros,974
previdenciário,974
internacional,974
ambiental,974
processual,974


In [4]:
from sklearn.preprocessing import LabelEncoder

# convertendo classes
encoder = LabelEncoder()
classes = encoder.fit_transform(classes)

print(classes[:10])
len(classes)

[3 0 0 1 3 7 0 3 1 7]


9740

In [5]:
# criação da lista de ementas

ementas = df[1]
print(ementas[:10])

constitucional  213470    nao ha qualqu inconstitucional formal lei n in...
administrativo  993055    fat autenticaco banc nao ser perfeit legiv tod...
                183316    aca mandamental pressupo prov preconstitu nao ...
ambiental       920317    crim pesc proib lei art inci depend norm suple...
constitucional  802969    cf exprim tod eficac jurid nel cont mostrandos...
previdenciário  275243    divergenc jurisprudencial comprov inteligenc a...
administrativo  243660    hipotes juiz quo garant agrav candidat inscrit...
constitucional  139928    nao cab superior tribunal justic aprec mat ind...
ambiental       27160     federal competenc process julg aca penal fund ...
previdenciário  995418    federal assegur assistenc social necessit inde...
Name: 1, dtype: object


In [6]:
# uso de word2vec para a extração de features

from gensim.models import Word2Vec

ementas_list = [word for word in ementas.iteritems()]
ementas_list_list = []
for doc in ementas_list:
    ementas_list_list.append(doc[1].split())

X = Word2Vec(ementas_list_list, min_count=2, window=5, workers=4)
X.wv.init_sims() # para inicializar model.vw.syn0norm, necessário para o cálculo da média da
print(X)
X.save('big_word2vec.model')

Word2Vec(vocab=9979, size=100, alpha=0.025)


In [12]:
# funções para calcular a média vetorial das palavras de cada ementa
import numpy as np


def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.vocab:
            mean.append(wv.vectors[wv.vocab[word].index])
            all_words.add(wv.vocab[word].index)

    if not mean:
        # print("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(wv.vector_size,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, post) for post in text_list ])

In [13]:
# função para tokenizar as palavras com frequência maior que 2
import nltk


def w2v_tokenize_text(text):    
    tokens = []
    for sent in nltk.sent_tokenize(text, language='portuguese'):
        for word in nltk.word_tokenize(sent, language='portuguese'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens

In [14]:
# Like any other supervised machine learning problem, we need to divide data into 
# 20% test set and 80% training set

import gensim
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec


train, test = train_test_split(df, test_size=0.2, random_state=0)

test_tokenized = test.apply(lambda r: w2v_tokenize_text(r[1]), axis=1).values
train_tokenized = train.apply(lambda r: w2v_tokenize_text(r[1]), axis=1).values

# X = Word2Vec.load('big_word2vec.model')
X_train = word_averaging_list(X.wv,train_tokenized)
X_test = word_averaging_list(X.wv,test_tokenized)

y_train = train[0]
y_test = test[0]

print("train: {},{}".format(len(X_train),len(y_train)))
print("test: {},{}".format(len(X_test),len(y_test)))

print(X_train)
print(X_test)

train: 7792,7792
test: 1948,1948


In [16]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix


# Modelos para treinar
names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators=1000, random_state=0, class_weight='balanced'),
    LogisticRegression(solver='lbfgs', multi_class='multinomial'),
    SGDClassifier(max_iter = 100, tol=1e-3),
    SVC(kernel = 'linear')
]

models = zip(names, classifiers)

for name, classifier in models:
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)*100
    print("{} Accuracy: {}".format(name, accuracy))

K Nearest Neighbors Accuracy: 44.455852156057496
Decision Tree Accuracy: 33.2135523613963
Random Forest Accuracy: 51.28336755646817


/home/audora/tcc/venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Logistic Regression Accuracy: 49.58932238193018
SGD Classifier Accuracy: 49.58932238193018
Naive Bayes Accuracy: 49.332648870636554


In [17]:
# Classificador por votos
from sklearn.ensemble import VotingClassifier

names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators=1000, random_state=0),
    LogisticRegression(solver='lbfgs', multi_class='multinomial'),
    SGDClassifier(max_iter = 100, tol=1e-3),
    SVC(kernel = 'linear')
]

models = list(zip(names, classifiers))
zipped_models_1 = models[:]
zipped_models_2 = list(models)


votingClassifier = VotingClassifier(estimators = zipped_models_2, voting = 'hard', n_jobs = -1)
votingClassifier.fit(X_train, y_train)
y_pred = votingClassifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)*100
print("Voting Classifier: Accuracy: {}".format(accuracy))

Voting Classifier: Accuracy: 51.48870636550308


In [18]:
# relatório de classificação

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

                precision    recall  f1-score   support

administrativo       0.36      0.32      0.34       195
     ambiental       0.70      0.73      0.72       203
         civil       0.43      0.38      0.40       192
constitucional       0.41      0.42      0.42       189
 internacional       0.63      0.62      0.63       195
        outros       0.55      0.48      0.52       190
         penal       0.60      0.60      0.60       193
previdenciário       0.60      0.67      0.63       199
    processual       0.30      0.35      0.32       196
    tributário       0.55      0.57      0.56       196

      accuracy                           0.51      1948
     macro avg       0.51      0.51      0.51      1948
  weighted avg       0.51      0.51      0.51      1948



In [19]:
# usando undersampling
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=0)
rus.fit(X_train, y_train)
X_resampled, y_resampled = rus._fit_resample(X_train, y_train)
print("train: {},{}".format(len(X_resampled),len(y_resampled)))


train: 7710,7710


In [ ]:
# Fazendo a mesma análisa usando undersampling

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix


# Modelos para treinar
names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators=1000, random_state=0, class_weight='balanced'),
    LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=200),
    SGDClassifier(max_iter = 100, tol=1e-3),
    SVC(kernel = 'linear')
]

models = zip(names, classifiers)

for name, classifier in models:
    classifier.fit(X_resampled, y_resampled)
    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)*100
    print("{} Accuracy: {}".format(name, accuracy))

In [ ]:
# Classificador por votos
from sklearn.ensemble import VotingClassifier

names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators=1000, random_state=0, class_weight='balanced'),
    LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=200),
    SGDClassifier(max_iter = 100, tol=1e-3),
    SVC(kernel = 'linear')
]

models = list(zip(names, classifiers))
zipped_models_1 = models[:]
zipped_models_2 = list(models)


votingClassifier = VotingClassifier(estimators = zipped_models_2, voting = 'hard', n_jobs = -1)
votingClassifier.fit(X_resampled, y_resampled)
y_pred = votingClassifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)*100
print("Voting Classifier: Accuracy: {}".format(accuracy))

In [ ]:
# relatório de classificação

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

#print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))